# Variational Quantum Algorithms with Qiskit Runtime: <br>Generic Example

In this notebook, we will work through a simple example of building variational quantum algorithms using Qiskit Runtime.

1. Define a Hamiltonian, representing system of interest, from Pauli operators
2. Select a quantum circuit representing the ground state ansatz
3. Construct an optimization loop to minimize the energy (expectation value of the Hamiltonian)
4. Retrieve the result (approximate ground state energy)


## 1. Define a (very simple) Hamiltonian

In [1]:
from qiskit.quantum_info import SparsePauliOp

op = SparsePauliOp.from_list([("ZZII", 1)])
num_qubits = op.num_qubits
ops = [op]

print(op)

SparsePauliOp(['ZZII'],
              coeffs=[1.+0.j])


#### This Hamiltonian has a minimum energy of $-1$ (which you can see from eigenvalues of $Z$)

In [2]:
target_energy = -1

## 2. Choose a ground state ansatz

We'll use a parametrized circuit from the Qiskit Circuit Library.<br>
[`EfficientSU2` yields short-depth circuits](https://qiskit.org/documentation/stubs/qiskit.circuit.library.EfficientSU2.html) well suited for noisy quantum hardware.

In [3]:
from qiskit.circuit.library import EfficientSU2
circuit = EfficientSU2(num_qubits, reps=1, entanglement="linear", insert_barriers=True)

circuit.decompose().draw()

┌──────────┐┌──────────┐ ░                 ░  ┌──────────┐┌───────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├─░───■─────────────░──┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐           ░  ├──────────┤├───────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├──■────────░──┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├
     ├──────────┤├──────────┤ ░ └───┘┌─┴─┐      ░ ┌┴──────────┤├───────────┤
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─░──────┤ X ├──■───░─┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├
     ├──────────┤├──────────┤ ░      └───┘┌─┴─┐ ░ ├───────────┤├───────────┤
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─░───────────┤ X ├─░─┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├
     └──────────┘└──────────┘ ░           └───┘ ░ └───────────┘└───────────┘

## 3. Construct an optimization loop to minimize energy

- Since we are interested in minimum energy (expval of Hamiltonian), we'll use the **Estimator primitive**.
- We'll choose a **classical optimizer** to update the circuit ansatz parameters and configure the settings of the Qiskit Runtime Service.
- We'll define a **callback function** to store extra info about the optimization.

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
from qiskit.algorithms.optimizers import SPSA
import numpy as np

service = QiskitRuntimeService()
options = {"backend" : "ibmq_qasm_simulator"}

# Callback: number of function evals, parameters, function value, stepsize, whether step was accepted
history = {"nfevs": [], "points": [], "fvals": [], "updates": [], "accepted": []}

#### Now, we use the Estimator primitive:

In [5]:
with Estimator(circuits=circuit, observables=ops, options=options, service=service) as estimator:
    
    def evaluate_expectation(x):
        x = list(x)
        results = estimator(circuits=[0], observables=[0], parameter_values=[x]).values[0]
        return np.real(results)  
    
    def callback(nfev, point, fval, update, accepted):
        print('expectation value: {}'.format(fval))
        history["nfevs"].append(nfev)
        history["points"].append(point)
        history["fvals"].append(fval)
        history["updates"].append(update)
        history["accepted"].append(accepted)

    initial_point = np.random.random(circuit.num_parameters)
    optimizer = SPSA(20, callback=callback)
    result = optimizer.minimize(fun=evaluate_expectation, x0=initial_point)


expectation value: -0.072265625
expectation value: -0.26171875
expectation value: -0.490234375
expectation value: -0.873046875
expectation value: -0.923828125
expectation value: -0.9921875
expectation value: -0.98046875
expectation value: -0.990234375
expectation value: -0.98828125
expectation value: -0.99609375
expectation value: -0.998046875
expectation value: -1.0
expectation value: -1.0
expectation value: -1.0
expectation value: -1.0
expectation value: -1.0
expectation value: -0.998046875
expectation value: -1.0
expectation value: -1.0
expectation value: -1.0


#### Our algorithm reached the target energy:

In [6]:
history["fvals"][-1] - target_energy

0.0

#### We can get other information about the optimization, such as the final circuit parameters:

In [7]:
print('optimal point:\n{}'.format(result.x))

optimal point:
[ 0.46522405 -0.52476735  1.83196964  3.13536883  0.38380442  2.09126234
  1.25463965  0.83967941  1.52898628  1.62002559  0.00551032 -0.01501302
  0.22517078  1.07463775  3.30407308  1.3216684 ]


In [8]:
import qiskit_ibm_runtime
qiskit_ibm_runtime.version.get_version_info()

'0.6.0'

In [9]:
from qiskit.tools.jupyter import *

%qiskit_version_table
%qiskit_copyright